In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%matplotlib inline
import matplotlib.pyplot as plt

##### lists of data in each folder: copy and paste into the cell below
* ./input/  
inputFolderFormat = "./input/{0}"
inputCsvList = ['QCOM.csv', 'GOOG.csv', 'NOK.csv', '^DJI.csv', '^TWII.csv', 
           '^N225.csv', '^GDAXI.csv', '^IXIC.csv', '^KS11.csv', '^SSEC.csv', 
           'NSU.DE_10_5_5.csv', 'DAI.DE_10_5_5.csv', 'BMW.DE_10_5_5.csv', 
           '0050.TW.csv', '1216.TW.csv', '1301.TW.csv', '2002.TW.csv', 
           '2330.TW.csv', '2409.TW.csv', '2454.TW.csv', '1704.TW.csv']

* ./input/2001_2010/  
inputFolderFormat = "./input/2001_2010/{0}"
inputCsvList = ['QCOM.csv', 'NOK.csv', '^DJI.csv', '^TWII.csv', 
            '^N225.csv', '^GDAXI.csv', '^IXIC.csv', '^KS11.csv', 
            '^SSEC.csv', 'NSU.DE.csv', 'DAI.DE.csv', 'BMW.DE.csv']  
            
* ./input/fall/  
inputFolderFormat = "./input/fall/{0}"  
inputCsvList = ['1314.TW.csv', '2323.TW.csv', '2349.TW.csv', 
'2353.TW.csv', '2405.TW.csv', '2409.TW.csv', '2601.TW.csv', 
'2885.TW.csv', '2888.TW.csv', '6116.TW.csv']         

In [2]:
from os import listdir

inputFolderFormat = "./input/temp/{0}"
inputCsvList = listdir('./input/temp')

full_action = True  # if True: buy with all cash, sell all stock
init_invest = 10000
consrange = 50     # consecutive days max. in the prob conslist

conslist = list()
for i in range(0, consrange*2 + 1):
    conslist.append([0, 0])

allrwdlist = list() 

data_result = list()

resourceCount = 0
    
for resource_data in inputCsvList:
    
    if resource_data.startswith('.'):
        continue
    
    resourceCount += 1
    print(resourceCount)
    
    for i in range(0, consrange*2 + 1):
        conslist[i][0] = 0
        conslist[i][1] = 0
    
    input_csv = inputFolderFormat.format(resource_data)
    data = pd.read_csv(input_csv)
    
    data.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
    data = data.drop(index=0).reset_index(drop=True)
    data = data.drop(index=0).reset_index(drop=True)
    data = data.iloc[::-1].reset_index(drop=True)
    data.to_csv('file_namePA.csv', sep=',', index=False)
    data = pd.read_csv('file_namePA.csv')
    
    close = data['Close']
    close = close.fillna(method='ffill')
    close = close.fillna(method='bfill')
    close = close.tolist()
    np.isnan(close).any()
    
    rwdlist = list()

    state = {'NumStock': 0, 'StockPrice': close[0], 'HandCash': init_invest}

    cn = 0 # consecutive day num

    for point in range(0, len(close)):
        
        #if point % 100 == 0:
        #    for i in range(0, len(conslist)):
        #        conslist[i][0] *= 0.5
        #        conslist[i][1] *= 0.5
        for i in range(0, len(conslist)):
            conslist[i][0] *= 0.99
            conslist[i][1] *= 0.99
        
        #if point%45 == 0:
        #    for i in range(0, consrange*2 + 1):
        #        conslist.append([0, 0])
        

        state['StockPrice'] = close[point]

        if point == 0:
            cn = 1 # serve as stok price going up
        elif 1 <= cn <= consrange: # consequtive goes up
            if close[point] >= close[point - 1]:
                conslist[cn][0] += 1
                cn += 1
            else:
                conslist[cn][1] += 1
                cn = consrange + 1
        elif consrange+1 <= cn <= consrange*2: # consequtive goes down
            if close[point] < close[point - 1]:
                conslist[cn][1] += 1
                cn += 1
            else:
                conslist[cn][0] += 1
                cn = 1

        tmp = cn

        # ar: action ratio
        while(1):          
            if conslist[tmp][0] + conslist[tmp][1] > 0:
                ar = conslist[tmp][0] / (conslist[tmp][0] + conslist[tmp][1]) # ar: prob of next price going up
                break
            else:
                tmp -= 1
                if tmp == 0 or tmp == consrange:
                    ar = 0.5
                    break

        if ar > 0.5: # next price may go up, let's buy
            ar = (ar - 0.5)*2

            if full_action:
                state['NumStock'] += state['HandCash'] / state['StockPrice']
                state['HandCash'] = 0
            else:
                state['NumStock'] += state['HandCash'] * ar / state['StockPrice']
                state['HandCash'] *= 1 - ar



        else: # next price may go down, let's sell
            ar = 1 - ar # ar: prob of next price going down
            ar = (ar - 0.5)*2

            if full_action:
                state['HandCash'] += state['NumStock'] * state['StockPrice']
                state['NumStock'] = 0
            else:
                state['HandCash'] += state['NumStock'] * ar * state['StockPrice']
                state['NumStock'] *= 1 - ar

        rwdlist.append(state['HandCash'] + state['StockPrice']*state['NumStock'] - 10000)
    allrwdlist.append(rwdlist)
                
    #print(resource_data, state['HandCash'] + state['StockPrice']*state['NumStock'] - 10000)
    #print(resource_data)
    
    data_result.append(state['HandCash'] + state['StockPrice']*state['NumStock'] - 10000)
print(data_result)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [3]:
print(data_result)

[-9076.750276234496, 16311.638164186454, -81.34655122833283, 35350.91309592402, -1540.4191135617657, 13272.83214763345, 11222.515682815341, 3251.040616148177, 16534.94173127873, 2863.5245666186656, -5201.559975341914, 4733.2171478669425, 1840668.9927826878, 75267.24276620215, 8463.744385530972, 3098.049134263516, 2203.092392535851, 24568.52974564842, -4613.372610682712, 1976.2532790113746, 6439.988103364754, 20700.345684373882, 23615.941471261205, -207.66530748340847, 150692.66688132475, 2781.7861690733916, -979.901587834438, -1848.2127906566293, -4690.733642427935, 12151.884517736442, 6117.556247315804, 5096.888447528068, 98344.18287334124, 6567.585098757871, 14684.393621755455, 32716.434555024323, -3576.0139717671973, 2120.1862935698846, 1115.7151920406104, 16488.20080958211, 47387.24017499269, 5105.173769673809, 6252.924254537333, 2763.4968819956666, 7944.166571948026, 48457.27317133902, 4421.442900010121, -4770.38452050992, -9129.732809714164, -7964.481519070582, 9073.483871457578,

#### conslist explanation
* **conslist[1] = [605, 332] ----> stock rose (one day in a row) include today**, in our previous experience, there were 605 days the tomorrow price raise, 332 days the tomorrow price fall
* conslist[2] ----> stock rose two days in a row include today
* ...
* conslist[consrange] ----> stock rose {consrange} days in a row include today
* conslist[consrange + 1] ----> stock fell (one day in a row) include today
* conslist[consrange + 2] ----> stock fell two days in a row include today
* ...
* conslist[consrange*2] ----> stock fell {consrange} days in a row include today
* **conslist[3] = [605, 332] ----> stock rose 3 days in a row include today. In our previous experience, there were 605 days the tomorrow price raise, 332 days the tomorrow price fall in the same situation(i.e. rose 3 days in a row).**

In [4]:
inputCsvList

['TMR.F 01-01-2001 - 31-12-2005 d.csv',
 'BIO 01-01-2001 - 31-12-2005 d.csv',
 'MCD 01-01-2001 - 31-12-2005 d.csv',
 'ACCEL.AS 01-01-2001 - 31-12-2005 d.csv',
 'MUB.BE 01-01-2001 - 31-12-2005 d.csv',
 'FRAF 01-01-2001 - 31-12-2005 d.csv',
 'P19.SI 01-01-2001 - 31-12-2005 d.csv',
 'MZP.F 01-01-2001 - 31-12-2005 d.csv',
 'SPX.L 01-01-2001 - 31-12-2005 d.csv',
 'EUCA.F 01-01-2001 - 31-12-2005 d.csv',
 'KARO.ST 01-01-2001 - 31-12-2005 d.csv',
 'CPM.MU 01-01-2001 - 31-12-2005 d.csv',
 'T19.SI 01-01-2001 - 31-12-2005 d.csv',
 'GFM.L 01-01-2001 - 31-12-2005 d.csv',
 'HOT.HM 01-01-2001 - 31-12-2005 d.csv',
 'JOX.MU 01-01-2001 - 31-12-2005 d.csv',
 'BRID 01-01-2001 - 31-12-2005 d.csv',
 'AWC.AX 01-01-2001 - 31-12-2005 d.csv',
 '7HP.MU 01-01-2001 - 31-12-2005 d.csv',
 'RHK.DU 01-01-2001 - 31-12-2005 d.csv',
 'EVN.BE 01-01-2001 - 31-12-2005 d.csv',
 'ANSS 01-01-2001 - 31-12-2005 d.csv',
 'SARKY.IS 01-01-2001 - 31-12-2005 d.csv',
 'CRLO.PA 01-01-2001 - 31-12-2005 d.csv',
 'SBS.F 01-01-2001 - 31-12